In [ ]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)

USE_COMPLEX = False  # <--- Set True ONLY if you need complex PETSc
USE_CLEAN = False    # <--- Set True to remove existing environment

opts_str = " ".join(
  [o for c, o in [(USE_COMPLEX, "--complex"), (USE_CLEAN, "--clean")] if c]
)

get_ipython().run_line_magic("run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts_str}")

🔧 FEniCSx Setup Configuration
PETSc type      : real
Clean install   : False

⚠️  Google Drive not mounted — using local cache (/content)

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Real PETSc (float64)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Real PETSc is installed
   - Recommended for most FEM problems
   - For complex problems, reinstall with --complex


---

In [ ]:
%%fenicsx

"""
Advanced UFL Elements Examples
===============================

More detailed examples for specific topics from the presentation.
These examples go deeper into each concept.

Based on FEniCS Workshop materials.
"""

import basix.ufl
import ufl
import numpy as np

print("=" * 70)
print("ADVANCED UFL ELEMENTS EXAMPLES")
print("=" * 70)

# =============================================================================
# Advanced Example 1: Working with Different Cell Types in Detail
# =============================================================================
print("\n" + "=" * 70)
print("Advanced Example 1: Detailed Cell Type Exploration")
print("=" * 70)

# 1D cells
print("\n1D CELLS:")
interval_elem = basix.ufl.element("Lagrange", "interval", 2, shape=(1,))
print(f"  Interval (1D): {interval_elem}")
print(f"    Use case: 1D problems, beam elements")

# 2D cells
print("\n2D CELLS:")
triangle_elem = basix.ufl.element("Lagrange", "triangle", 2, shape=(2,))
print(f"  Triangle: {triangle_elem}")
print(f"    Use case: Unstructured 2D meshes, flexible geometry")

quad_elem = basix.ufl.element("Lagrange", "quadrilateral", 2, shape=(2,))
print(f"  Quadrilateral: {quad_elem}")
print(f"    Use case: Structured meshes, tensor product spaces")

# 3D cells
print("\n3D CELLS:")
tet_elem = basix.ufl.element("Lagrange", "tetrahedron", 2, shape=(3,))
print(f"  Tetrahedron: {tet_elem}")
print(f"    Use case: Most flexible 3D mesh, complex geometries")

hex_elem = basix.ufl.element("Lagrange", "hexahedron", 2, shape=(3,))
print(f"  Hexahedron: {hex_elem}")
print(f"    Use case: Structured 3D meshes, better for some solvers")

prism_elem = basix.ufl.element("Lagrange", "prism", 2, shape=(3,))
print(f"  Prism (wedge): {prism_elem}")
print(f"    Use case: Layered domains, extrusions")

pyramid_elem = basix.ufl.element("Lagrange", "pyramid", 2, shape=(3,))
print(f"  Pyramid: {pyramid_elem}")
print(f"    Use case: Transitioning between tet and hex meshes")

# =============================================================================
# Advanced Example 2: Higher-Order Geometry (Curved Elements)
# =============================================================================
print("\n" + "=" * 70)
print("Advanced Example 2: Higher-Order Geometry for Curved Boundaries")
print("=" * 70)

# Linear geometry (straight edges)
linear_coord = basix.ufl.element("Lagrange", "triangle", 1, shape=(2,))
print(f"Linear coordinate element (P1): {linear_coord}")
print(f"  → Straight edges only")
print(f"  → Good for simple geometries")

# Quadratic geometry (curved edges)
quadratic_coord = basix.ufl.element("Lagrange", "triangle", 2, shape=(2,))
print(f"\nQuadratic coordinate element (P2): {quadratic_coord}")
print(f"  → Curved edges possible")
print(f"  → Better for circles, cylinders, etc.")

# Cubic geometry (highly curved)
cubic_coord = basix.ufl.element("Lagrange", "triangle", 3, shape=(2,))
print(f"\nCubic coordinate element (P3): {cubic_coord}")
print(f"  → Highly curved edges")
print(f"  → Accurate representation of complex boundaries")

# Example: Curved domain with higher-order solution
domain_curved = ufl.Mesh(quadratic_coord)
solution_element = basix.ufl.element("Lagrange", "triangle", 4, shape=(2,))
print(f"\nExample combination:")
print(f"  Geometry: P2 (curved)")
print(f"  Solution: P4 (sub-parametric)")
print(f"  → High accuracy on curved domains")

# =============================================================================
# Advanced Example 3: Complex Mixed Element Spaces
# =============================================================================
print("\n" + "=" * 70)
print("Advanced Example 3: Complex Mixed Element Formulations")
print("=" * 70)

cell = "triangle"

# Example 1: Thermoelasticity (displacement + temperature)
print("\n1. THERMOELASTICITY:")
displacement_el = basix.ufl.element("Lagrange", cell, 2, shape=(2,))
temperature_el = basix.ufl.element("Lagrange", cell, 1)
thermoelastic_el = basix.ufl.mixed_element([displacement_el, temperature_el])
print(f"  Displacement (u): P2 vector")
print(f"  Temperature (T): P1 scalar")
print(f"  Mixed element: {thermoelastic_el}")

# Example 2: Poroelasticity (displacement + pressure)
print("\n2. POROELASTICITY:")
u_el = basix.ufl.element("Lagrange", cell, 2, shape=(2,))
p_el = basix.ufl.element("Lagrange", cell, 1)
poroelastic_el = basix.ufl.mixed_element([u_el, p_el])
print(f"  Displacement: P2 vector")
print(f"  Pressure: P1 scalar")
print(f"  Mixed element: {poroelastic_el}")

# Example 3: Mixed Poisson (flux + potential)
print("\n3. MIXED POISSON:")
flux_el = basix.ufl.element("RT", cell, 1)  # Raviart-Thomas
potential_el = basix.ufl.element("DG", cell, 0)
mixed_poisson_el = basix.ufl.mixed_element([flux_el, potential_el])
print(f"  Flux (σ): RT1 (Raviart-Thomas)")
print(f"  Potential (u): DG0 (piecewise constant)")
print(f"  Mixed element: {mixed_poisson_el}")

# Example 4: Three-field formulation
print("\n4. THREE-FIELD FORMULATION (Example):")
field1_el = basix.ufl.element("Lagrange", cell, 2, shape=(2,))
field2_el = basix.ufl.element("Lagrange", cell, 1)
field3_el = basix.ufl.element("DG", cell, 0)
three_field_el = basix.ufl.mixed_element([field1_el, field2_el, field3_el])
print(f"  Field 1: P2 vector")
print(f"  Field 2: P1 scalar")
print(f"  Field 3: DG0 scalar")
print(f"  Mixed element: {three_field_el}")

# =============================================================================
# Advanced Example 4: Element Enrichment Strategies
# =============================================================================
print("\n" + "=" * 70)
print("Advanced Example 4: Different Enrichment Strategies")
print("=" * 70)

# Standard enrichment: P1 + Bubble
print("\n1. STANDARD P1 + BUBBLE:")
p1_el = basix.ufl.element("Lagrange", cell, 1)
bubble_el = basix.ufl.element("Bubble", cell, 3)
p1_bubble = basix.ufl.enriched_element([p1_el, bubble_el])
print(f"  Base: P1 Lagrange")
print(f"  Enrichment: Cubic bubble")
print(f"  Result spans: {{1, x, y, xy(1-x-y)}}")

# P2 + Bubble for more accuracy
print("\n2. P2 + BUBBLE:")
p2_el = basix.ufl.element("Lagrange", cell, 2)
p2_bubble = basix.ufl.enriched_element([p2_el, bubble_el])
print(f"  Base: P2 Lagrange")
print(f"  Enrichment: Cubic bubble")
print(f"  Result: Higher-order accurate space")

# Multiple enrichments
print("\n3. MULTIPLE ENRICHMENTS:")
p1_el = basix.ufl.element("Lagrange", cell, 1)
bubble3_el = basix.ufl.element("Bubble", cell, 3)
bubble4_el = basix.ufl.element("Bubble", cell, 4)
multi_enriched = basix.ufl.enriched_element([p1_el, bubble3_el, bubble4_el])
print(f"  Base: P1")
print(f"  Enrichment 1: Cubic bubble")
print(f"  Enrichment 2: Quartic bubble")

# =============================================================================
# Advanced Example 5: Blocked Elements for Vector Spaces
# =============================================================================
print("\n" + "=" * 70)
print("Advanced Example 5: Blocked Elements")
print("=" * 70)

# Create enriched element first
enriched = basix.ufl.enriched_element([
    basix.ufl.element("Lagrange", cell, 2),
    basix.ufl.element("Bubble", cell, 3)
])

# Block to 2D vector
blocked_2d = basix.ufl.blocked_element(enriched, shape=(2,))
print(f"2D blocked element: {blocked_2d}")
print(f"  Use: 2D velocity fields in fluids")

# Block to 3D vector
blocked_3d = basix.ufl.blocked_element(enriched, shape=(3,))
print(f"\n3D blocked element: {blocked_3d}")
print(f"  Use: 3D velocity fields, displacement")

# Block to custom dimensions
blocked_custom = basix.ufl.blocked_element(enriched, shape=(4,))
print(f"\nCustom blocked element (4D): {blocked_custom}")
print(f"  Use: Special applications requiring 4 components")

# =============================================================================
# Advanced Example 6: DG Elements of Different Orders
# =============================================================================
print("\n" + "=" * 70)
print("Advanced Example 6: Discontinuous Galerkin Elements (Various Orders)")
print("=" * 70)

for degree in range(0, 5):
    dg_el = basix.ufl.element("DG", cell, degree)
    print(f"DG{degree}: {dg_el}")

    if degree == 0:
        print(f"  → Piecewise constant (1 DOF per cell)")
    elif degree == 1:
        print(f"  → Piecewise linear (3 DOFs per triangle)")
    elif degree == 2:
        print(f"  → Piecewise quadratic (6 DOFs per triangle)")
    else:
        print(f"  → Higher-order DG")

# Vector DG element
print("\nVector DG element:")
vector_dg = basix.ufl.element("DG", cell, 2, shape=(2,))
print(f"  {vector_dg}")
print(f"  Use: DG methods for vector fields (velocity, etc.)")

# =============================================================================
# Advanced Example 7: Comparison of Stable Mixed Elements
# =============================================================================
print("\n" + "=" * 70)
print("Advanced Example 7: Stable Mixed Elements for Stokes")
print("=" * 70)

# Taylor-Hood P2-P1
print("\n1. TAYLOR-HOOD (P2-P1):")
th_v = basix.ufl.element("Lagrange", cell, 2, shape=(2,))
th_p = basix.ufl.element("Lagrange", cell, 1)
taylor_hood = basix.ufl.mixed_element([th_v, th_p])
print(f"  Velocity: P2")
print(f"  Pressure: P1")
print(f"  Properties: High accuracy, more DOFs")

# Taylor-Hood P3-P2 (higher order)
print("\n2. TAYLOR-HOOD (P3-P2):")
th_v_p3 = basix.ufl.element("Lagrange", cell, 3, shape=(2,))
th_p_p2 = basix.ufl.element("Lagrange", cell, 2)
taylor_hood_p3 = basix.ufl.mixed_element([th_v_p3, th_p_p2])
print(f"  Velocity: P3")
print(f"  Pressure: P2")
print(f"  Properties: Very high accuracy, many DOFs")

# MINI element
print("\n3. MINI ELEMENT:")
mini_v_base = basix.ufl.element("Lagrange", cell, 1)
mini_bubble = basix.ufl.element("Bubble", cell, 3)
mini_enriched = basix.ufl.enriched_element([mini_v_base, mini_bubble])
mini_v = basix.ufl.blocked_element(mini_enriched, shape=(2,))
mini_p = basix.ufl.element("Lagrange", cell, 1)
mini = basix.ufl.mixed_element([mini_v, mini_p])
print(f"  Velocity: P1 + Bubble")
print(f"  Pressure: P1")
print(f"  Properties: Fewer DOFs, sufficient accuracy")

# Crouzeix-Raviart
print("\n4. CROUZEIX-RAVIART (P2-P0):")
cr_v = basix.ufl.element("Lagrange", cell, 2, shape=(2,))
cr_p = basix.ufl.element("DG", cell, 0)
crouzeix_raviart = basix.ufl.mixed_element([cr_v, cr_p])
print(f"  Velocity: P2")
print(f"  Pressure: P0 (DG)")
print(f"  Properties: Discontinuous pressure")

# =============================================================================
# Advanced Example 8: Elements for Specific PDEs
# =============================================================================
print("\n" + "=" * 70)
print("Advanced Example 8: Element Selection for Different PDEs")
print("=" * 70)

examples = {
    "Poisson/Heat Equation": {
        "element": basix.ufl.element("Lagrange", cell, 2),
        "description": "Simple scalar Lagrange, any degree"
    },
    "Linear Elasticity": {
        "element": basix.ufl.element("Lagrange", cell, 2, shape=(2,)),
        "description": "Vector Lagrange for displacement"
    },
    "Stokes Flow": {
        "element": basix.ufl.mixed_element([
            basix.ufl.element("Lagrange", cell, 2, shape=(2,)),
            basix.ufl.element("Lagrange", cell, 1)
        ]),
        "description": "Taylor-Hood (P2-P1) mixed element"
    },
    "Maxwell Equations": {
        "element": basix.ufl.element("N1curl", cell, 1),
        "description": "Nédélec edge elements"
    },
    "Mixed Poisson": {
        "element": basix.ufl.mixed_element([
            basix.ufl.element("RT", cell, 1),
            basix.ufl.element("DG", cell, 0)
        ]),
        "description": "Raviart-Thomas + DG"
    },
    "Advection-Diffusion (DG)": {
        "element": basix.ufl.element("DG", cell, 2),
        "description": "DG for hyperbolic problems"
    }
}

for pde_name, info in examples.items():
    print(f"\n{pde_name}:")
    print(f"  Element: {info['element']}")
    print(f"  Reason: {info['description']}")

# =============================================================================
# Advanced Example 9: Element Hierarchy Demonstration
# =============================================================================
print("\n" + "=" * 70)
print("Advanced Example 9: Building Complex Elements Step by Step")
print("=" * 70)

print("\nBuilding a MINI element from scratch:")
print("\nStep 1: Create base Lagrange element")
step1 = basix.ufl.element("Lagrange", cell, 1)
print(f"  {step1}")

print("\nStep 2: Create bubble element")
step2 = basix.ufl.element("Bubble", cell, 3)
print(f"  {step2}")

print("\nStep 3: Enrich Lagrange with bubble")
step3 = basix.ufl.enriched_element([step1, step2])
print(f"  {step3}")

print("\nStep 4: Block to vector (2D)")
step4 = basix.ufl.blocked_element(step3, shape=(2,))
print(f"  {step4}")

print("\nStep 5: Create pressure element")
step5 = basix.ufl.element("Lagrange", cell, 1)
print(f"  {step5}")

print("\nStep 6: Combine into mixed element")
step6 = basix.ufl.mixed_element([step4, step5])
print(f"  {step6}")

print("\n→ Final MINI element created!")

# =============================================================================
# Summary
# =============================================================================
print("\n" + "=" * 70)
print("SUMMARY OF ADVANCED CONCEPTS")
print("=" * 70)
print("""
Advanced Topics Covered:
1. Detailed exploration of all cell types
2. Higher-order geometry for curved boundaries
3. Complex mixed formulations (3+ fields)
4. Various enrichment strategies
5. Blocked elements for vector spaces
6. DG elements of different orders
7. Stable mixed elements comparison
8. PDE-specific element selection
9. Step-by-step element construction

Key Takeaways:
- Choose elements based on PDE physics
- Higher-order geometry for curved domains
- Mixed elements need stability conditions
- DG for advection-dominated problems
- Enrichment for specialized behaviors
- Taylor-Hood and MINI both work for Stokes
- Build complex elements incrementally
""")

print("\n" + "=" * 70)
print("Advanced examples completed successfully!")
print("=" * 70)

ADVANCED UFL ELEMENTS EXAMPLES

Advanced Example 1: Detailed Cell Type Exploration

1D CELLS:
  Interval (1D): blocked element (Basix element (P, interval, 2, gll_warped, unset, False, float64, []), (1,))
    Use case: 1D problems, beam elements

2D CELLS:
  Triangle: blocked element (Basix element (P, triangle, 2, gll_warped, unset, False, float64, []), (2,))
    Use case: Unstructured 2D meshes, flexible geometry
  Quadrilateral: blocked element (Basix element (P, quadrilateral, 2, gll_warped, unset, False, float64, []), (2,))
    Use case: Structured meshes, tensor product spaces

3D CELLS:
  Tetrahedron: blocked element (Basix element (P, tetrahedron, 2, gll_warped, unset, False, float64, []), (3,))
    Use case: Most flexible 3D mesh, complex geometries
  Hexahedron: blocked element (Basix element (P, hexahedron, 2, gll_warped, unset, False, float64, []), (3,))
    Use case: Structured 3D meshes, better for some solvers
  Prism (wedge): blocked element (Basix element (P, prism, 2,